In [1]:
!pip install tqdm

__Final Data Processing Code__

In [2]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import logging
import re
import os
import csv

# Define the management patterns
management_pattern = re.compile(r'ITEM\s+7\.?\s+MANAGEMENT\'?\s*S?(.*?)ITEM\s+[7A|7(a)|8]', re.IGNORECASE|re.DOTALL|re.MULTILINE)
# fallback_pattern = re.compile(r'MANAGEMENT\'?\s*S?\s*DISCUSSION(.*?)(?:ITEM\s+[7A|7(a)|8]|\Z)', re.IGNORECASE|re.DOTALL|re.MULTILINE)

# Define a pattern to check for other items
other_items_pattern = re.compile(r'\b(?:Item|ITEM)\s+(?!7\b)\d+', re.IGNORECASE)

def extract_company_name(content):
    """
    Extracts the company name from the file content.

    Parameters:
    - content (str): The full text of the 10-K document.

    Returns:
    - str: The extracted company name or "Unknown" if not found.
    """

    match = re.search(r'COMPANY CONFORMED NAME:\s*(.+)$', content, re.MULTILINE)
    return match.group(1).strip() if match else "Unknown"

def extract_filing_date(content):
    """
    Extracts the filing date from the file content.

    Parameters:
    - content (str): The full text of the 10-K document.

    Returns:
    - str: The extracted filing date in YYYY-MM-DD format or "Unknown" if not found.
    """

    match = re.search(r'FILED AS OF DATE:\s*(\d{8})', content)
    if match:
        date_str = match.group(1)
        return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
    return "Unknown"

def extract_company_id(content):
    """
    Extracts the company identifier (CIK) from the file content.

    Parameters:
    - content (str): The content of the file.

    Returns:
    - str or None: Extracted CIK or None if not found.
    """

    # Look for the CIK in the header of the file
    cik_pattern = r'CENTRAL INDEX KEY:\s*(\d{10})'
    match = re.search(cik_pattern, content)
    if match:
        return match.group(1)
    
    # If not found in the standard location, try an alternative pattern
    alt_pattern = r'CIK=(\d{10})'
    match = re.search(alt_pattern, content)
    if match:
        return match.group(1)
    
    return None

def count_companies(content):
    """
    Counts the number of unique companies in the SEC filing based on CIK numbers,
    company names, and filer identifiers.

    Parameters:
    - content (str): The full text of the 10-K document.

    Returns:
    - int: Number of unique companies detected.
    """

    # Extract CIK numbers using the existing helper function
    cik_number = extract_company_id(content)
    cik_numbers = set([cik_number]) if cik_number else set()

    # Extract company names using the existing helper function
    company_name = extract_company_name(content)
    company_names = set([company_name]) if company_name != "Unknown" else set()

    # Search for filer identifiers
    filer_pattern = r'FILER:\s*\n\s*COMPANY DATA:'
    filer_count = len(re.findall(filer_pattern, content))

    # Determine the number of unique companies
    num_companies = max(len(cik_numbers), len(company_names), filer_count)

    return num_companies

def contains_other_items(mda_content):
    """
    Checks if the MDA content contains mentions of items other than Item 7 more than two times.

    Parameters:
    - mda_content (str): The content of the MDA section.

    Returns:
    - bool: True if other items are mentioned more than twice, False otherwise.
    """
    matches = other_items_pattern.findall(mda_content)
    return len(matches) > 2

def check_management_pattern(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        
        # Check for management pattern using findall
        management_matches = management_pattern.findall(content)
        
        # Filter out matches with less than 500 words and check for other items
        valid_matches = []
        for match in management_matches:
            word_count = len(match.split())
            if word_count >= 500 and not contains_other_items(match):
                valid_matches.append((match, word_count))
        
        # If no valid matches found, try fallback pattern
        # if not valid_matches:
        #     fallback_matches = fallback_pattern.findall(content)
        #     for match in fallback_matches:
        #         word_count = len(match.split())
        #         if word_count >= 500 and not contains_other_items(match):
        #             valid_matches.append((match, word_count))
        
        if not valid_matches:
            return {
                'file_path': file_path,
                'has_management_pattern': False,
                'longest_match': "",
                'word_count': 0,
                'has_other_items': False
            }
        
        # Get the longest match
        longest_match = max(valid_matches, key=lambda x: x[1])
        
        # Get 50 words before the longest match
        try:
            match_start = content.index(longest_match[0])
            words_before = ' '.join(content[:match_start].split()[-50:])
        except ValueError:
            words_before = ''
        
        prefix = ' '.join(words_before.split())
        
        return {
            'file_path': file_path,
            'has_management_pattern': True,
            'longest_match': prefix + ' ' + longest_match[0],
            'word_count': longest_match[1],
            'has_other_items': False
        }

def pair_reports(results):
    """
    Pairs reports for companies based on their filing dates, allowing for delays and differences in publication dates.

    Parameters:
    - results (list of dict): Extracted MDA sections with filenames and filing dates.

    Returns:
    - list of dict: Paired reports.
    """
    # Group reports by company
    company_reports = {}
    for report in results:
        cik = report['cik_number']
        if cik not in company_reports:
            company_reports[cik] = []
        company_reports[cik].append(report)

    paired_reports = []
    for cik, reports in company_reports.items():
        # Sort reports by filing date
        sorted_reports = sorted(reports, key=lambda x: datetime.strptime(x['filing_date'], '%Y-%m-%d'))
        
        for i in range(len(sorted_reports) - 1):
            current_report = sorted_reports[i]
            next_report = sorted_reports[i + 1]
            
            current_date = datetime.strptime(current_report['filing_date'], '%Y-%m-%d')
            next_date = datetime.strptime(next_report['filing_date'], '%Y-%m-%d')
            
            # Check if reports are within a reasonable time frame (e.g., 9-15 months apart)
            if timedelta(days=270) <= next_date - current_date <= timedelta(days=450):
                paired_reports.append({
                    'current_filename': current_report['filename'],
                    'next_filename': next_report['filename'],
                    'company_name': current_report['company_name'],
                    'cik_number': cik,
                    'current_filing_date': current_report['filing_date'],
                    'next_filing_date': next_report['filing_date'],
                    'current_mda_content': current_report['mda_content'],
                    'next_mda_content': next_report['mda_content'],
                    'time_difference': (next_date - current_date).days
                })

    return paired_reports

def pair_and_save_reports(results, output_directory):
    """
    Pairs reports and saves them to a CSV file.

    Parameters:
    - results (list of dict): Extracted MDA sections with filenames and filing dates.
    - output_directory (str): Directory to save the paired reports CSV.

    Returns:
    - None
    """

    paired_reports_csv = os.path.join(output_directory, 'paired_mda_reports.csv')
    
    # Use the new pair_reports function that doesn't rely on fiscal_calendar_df
    paired_reports = pair_reports(results)
    
    if paired_reports:
        fieldnames = ['current_filename', 'next_filename', 'company_name', 'cik_number', 
                      'current_filing_date', 'next_filing_date', 'current_mda_content', 
                      'next_mda_content', 'time_difference']
        
        with open(paired_reports_csv, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in paired_reports:
                writer.writerow(row)
        logging.info(f"INFO: Paired reports saved to {paired_reports_csv}")
    else:
        logging.warning("WARNING: No paired reports found.")

def load_cik_ticker_mapping(json_path):
    """
    Loads the CIK to ticker mapping from a JSON file.

    Parameters:
    - json_path (str): Path to the cik_ticker_mapping.json file.

    Returns:
    - dict: Mapping of CIKs to tickers.
    """
    with open(json_path, 'r') as file:
        cik_ticker = json.load(file)
    return cik_ticker

def get_financial_variables(cik, year):
    """
    Retrieves financial variables for a given company and year.

    Parameters:
    - cik (str): Company Identifier Code.
    - year (int): Year for which to retrieve financial data.

    Returns:
    - dict: Financial variables.
    """
    # Placeholder implementation
    # Replace this with actual data retrieval logic
    financial_data = {
        'dividend_yield': np.random.rand(),
        'valuation': np.random.rand(),
        'growth': np.random.rand(),
        'profitability': np.random.rand(),
        'medium_term_momentum': np.random.rand(),
        'short_term_reversal': np.random.rand(),
        'volatility': np.random.rand(),
        'leverage': np.random.rand(),
        'liquidity': np.random.rand(),
        'size': np.random.rand()
    }
    return financial_data

def get_daily_prices(cik, year):
    """
    Retrieves daily closing prices for a given company and year.

    Parameters:
    - cik (str): Company Identifier Code.
    - year (int): Year for which to retrieve price data.

    Returns:
    - list: List of daily closing prices.
    """
    # Placeholder implementation
    # Replace this with actual data retrieval logic (e.g., from a database or API)
    np.random.seed(int(cik[-4:]))  # For reproducibility
    daily_prices = np.random.uniform(low=10, high=100, size=252).tolist()  # Approx. trading days in a year
    return daily_prices

def compute_mdd(prices):
    """
    Compute the Maximum Drawdown (MDD) for a series of prices.

    Parameters:
    - prices (list): List of daily closing prices.

    Returns:
    - float: Maximum Drawdown value.
    """
    peak = prices[0]
    max_drawdown = 0
    for price in prices:
        if price > peak:
            peak = price
        drawdown = (peak - price) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

def main():
    base_directory = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Mapped_Files'
    output_file = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\longest_mdas.txt'
    paired_reports_csv = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports_FINAL.csv'

    # Step 1: Extract and Pair MDA Sections
    all_files = []
    for root, dirs, files in os.walk(base_directory):
        for file in files:
            if file.lower().endswith('.txt') and ('10-k' in file.lower() or '10-k405' in file.lower()) and not any(x in file.lower() for x in ['10-k/a', '10-k-a', '10-k405-a']):
                all_files.append(os.path.join(root, file))

    max_workers = min(32, os.cpu_count() + 4)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(check_management_pattern, file_path): file_path for file_path in all_files}
        
        all_results = []
        for future in tqdm(as_completed(future_to_file), total=len(all_files), desc="Processing files"):
            file_path = future_to_file[future]
            try:
                result = future.result(timeout=60)
                if result['has_management_pattern']:
                    # Extract additional information
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                    company_name = extract_company_name(content)
                    cik_number = extract_company_id(content)
                    filing_date = extract_filing_date(content)
                    all_results.append({
                        'filename': os.path.basename(file_path),
                        'company_name': company_name,
                        'cik_number': cik_number,
                        'filing_date': filing_date,
                        'mda_content': result['longest_match']
                    })
            except Exception as exc:
                logging.error(f"ERROR: {file_path} generated an exception: {exc}")

    # Save paired reports
    if all_results:
        # Create DataFrame from all_results
        mda_df = pd.DataFrame(all_results)
        # Extract year from filing_date
        mda_df['year'] = pd.to_datetime(mda_df['filing_date'], errors='coerce').dt.year
        # Remove entries with invalid dates
        mda_df = mda_df.dropna(subset=['year'])
        
        # Save paired_reports_csv using existing function
        pair_and_save_reports(all_results, os.path.dirname(paired_reports_csv))
    else:
        print("No valid MDA data available for processing.")

    # Log the results
    with open(output_file, 'w', encoding='utf-8') as f:
        for result in all_results:
            f.write(f"File: {result['filename']}\n")
            f.write(f"Company: {result['company_name']}\n")
            f.write(f"CIK: {result['cik_number']}\n")
            f.write(f"Filing Date: {result['filing_date']}\n")
            f.write(f"Word count: {len(result['mda_content'].split())}\n")
            f.write(f"Contains other items more than twice: No\n\n")
            f.write(result['mda_content'].strip())
            f.write("\n\n---------------\n\n")

    print(f"Total files processed: {len(all_files)}")
    print(f"Files with valid management pattern: {len(all_results)}")
    print(f"Longest MDAs saved to: {output_file}")
    print(f"Paired reports saved to: {paired_reports_csv}")

if __name__ == "__main__":
    main()

Processing files: 100%|██████████| 54551/54551 [25:33<00:00, 35.57it/s]  


Total files processed: 54551
Files with valid management pattern: 34789
Longest MDAs saved to: C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\longest_mdas.txt
Paired reports saved to: C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports_FINAL.csv


In [2]:
import pandas as

# Read the CSV in chunks
chunk_size = 1000  # Adjust this value based on your system's memory
longest_current_mda = {'length': 0}
longest_next_mda = {'length': 0}

for chunk in pd.read_csv(r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports.csv', chunksize=chunk_size):
    # Calculate lengths for current chunk
    chunk['current_mda_length'] = chunk['current_mda_content'].str.split().str.len()
    chunk['next_mda_length'] = chunk['next_mda_content'].str.split().str.len()
    
    # Update longest current MDA if necessary
    current_max_idx = chunk['current_mda_length'].idxmax()
    if chunk.loc[current_max_idx, 'current_mda_length'] > longest_current_mda['length']:
        longest_current_mda = {
            'company': chunk.loc[current_max_idx, 'company_name'],
            'length': chunk.loc[current_max_idx, 'current_mda_length'],
            'content': chunk.loc[current_max_idx, 'current_mda_content']
        }
    
    # Update longest next MDA if necessary
    next_max_idx = chunk['next_mda_length'].idxmax()
    if chunk.loc[next_max_idx, 'next_mda_length'] > longest_next_mda['length']:
        longest_next_mda = {
            'company': chunk.loc[next_max_idx, 'company_name'],
            'length': chunk.loc[next_max_idx, 'next_mda_length'],
            'content': chunk.loc[next_max_idx, 'next_mda_content']
        }

print("\nLongest current MDA:")
print(f"Company: {longest_current_mda['company']}")
print(f"Length: {longest_current_mda['length']} words")
print("\nContent:")
print(longest_current_mda['content'])

print("\nLongest next MDA:")
print(f"Company: {longest_next_mda['company']}")
print(f"Length: {longest_next_mda['length']} words") 
print("\nContent:")
print(longest_next_mda['content'])

# Save current MDA content to a text file
output_path = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\longest_current_mda.txt'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(f"Company: {longest_current_mda['company']}\n")
    f.write(f"Length: {longest_current_mda['length']} words\n\n")
    f.write(longest_current_mda['content'])
print(f"\nLongest current MDA saved to: {output_path}")


Longest current MDA:
Company: METLIFE INC
Length: 103521 words

Content:
principle, net of income tax, of 10 million resulted from the adoption of SFAS 159, which decreased accumulated other comprehensive income (loss) at January 1, 2008. (7) Return on common equity is defined as net income available to common shareholders divided by average common stockholders equity. Item 7. Management s 
 Discussion and Analysis of Financial Condition and Results of
 Operations 

For purposes of this discussion, MetLife or the
 Company refers to MetLife, Inc., a Delaware
 corporation incorporated in 1999 (the Holding
 Company ), and its subsidiaries, including Metropolitan
 Life Insurance Company MLIC ). Following this
 summary is a discussion addressing the consolidated results of
 operations and financial condition of the Company for the
 periods indicated. This discussion should be read in conjunction
 with the forward-looking statement information included below,
 Risk Factors, Selected Financi

In [1]:
import pandas as pd
import os

# Use a smaller chunk size
chunk_size = 100  # Reduced from 1000

# First pass: identify duplicates
current_mdas = set()
next_mdas = set()
duplicate_current = set()
duplicate_next = set()

print("Starting first pass: identifying duplicates...")

# Add low_memory=False and specify dtypes for efficiency
dtype_dict = {
    'current_mda_content': str,
    'next_mda_content': str,
    'company_name': str,
    'cik_number': str,
    'current_filing_date': str,
    'next_filing_date': str
}

for chunk in pd.read_csv(r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports.csv', 
                        chunksize=chunk_size, 
                        low_memory=False,
                        dtype=dtype_dict):
    # Process current MDAs
    for idx, current_mda in chunk['current_mda_content'].items():
        if current_mda in current_mdas:
            duplicate_current.add(current_mda)
        else:
            current_mdas.add(current_mda)
    
    # Process next MDAs
    for idx, next_mda in chunk['next_mda_content'].items():
        if next_mda in next_mdas:
            duplicate_next.add(next_mda)
        else:
            next_mdas.add(next_mda)
    
    # Clear some memory
    del chunk

print("First pass complete. Starting second pass: removing duplicates...")

# Second pass: remove rows with duplicates
output_file = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports_CLEANED.csv'
if os.path.exists(output_file):
    os.remove(output_file)

first_chunk = True
rows_kept = 0
rows_removed = 0

for chunk in pd.read_csv(r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports.csv', 
                        chunksize=chunk_size,
                        low_memory=False,
                        dtype=dtype_dict):
    # Keep rows where neither MDA is in the duplicate sets
    clean_chunk = chunk[~(chunk['current_mda_content'].isin(duplicate_current) | 
                         chunk['next_mda_content'].isin(duplicate_next))]
    
    rows_kept += len(clean_chunk)
    rows_removed += len(chunk) - len(clean_chunk)
    
    # Write to output file
    if len(clean_chunk) > 0:
        clean_chunk.to_csv(output_file, mode='a', header=first_chunk, index=False)
    first_chunk = False
    
    # Clear some memory
    del chunk
    del clean_chunk

print(f"\nCleaning complete!")
print(f"Rows kept: {rows_kept}")
print(f"Rows removed: {rows_removed}")

print("\nStarting verification...")

# Verify the cleaning worked
current_mdas = set()
next_mdas = set()
duplicate_current = set()
duplicate_next = set()

for chunk in pd.read_csv(output_file, 
                        chunksize=chunk_size,
                        low_memory=False,
                        dtype=dtype_dict):
    for idx, current_mda in chunk['current_mda_content'].items():
        if current_mda in current_mdas:
            duplicate_current.add(current_mda)
        else:
            current_mdas.add(current_mda)
    
    for idx, next_mda in chunk['next_mda_content'].items():
        if next_mda in next_mdas:
            duplicate_next.add(next_mda)
        else:
            next_mdas.add(next_mda)
            
    # Clear some memory
    del chunk

print(f"\nVerification results:")
print(f"Number of duplicate current MDAs: {len(duplicate_current)}")
print(f"Number of duplicate next MDAs: {len(duplicate_next)}")

Starting first pass: identifying duplicates...
First pass complete. Starting second pass: removing duplicates...

Cleaning complete!
Rows kept: 27508
Rows removed: 9

Starting verification...

Verification results:
Number of duplicate current MDAs: 0
Number of duplicate next MDAs: 0


In [6]:
# Quick verification of the new file
file_path = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports.csv'

total_rows = 0
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    total_rows += len(chunk)
print(f"\nTotal rows in deduplicated file: {total_rows}")


Total rows in deduplicated file: 27517


In [1]:
import pandas as pd

# Count files from 2001-2009
cleaned_file = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports_CLEANED.csv'
df = pd.read_csv(cleaned_file)

# Extract years from filenames
df['current_year'] = df['current_filename'].str.extract(r'(\d{4})').astype(int)
df['next_year'] = df['next_filename'].str.extract(r'(\d{4})').astype(int)

# Count files in date range
files_2001_2009 = df[
    ((df['current_year'] >= 2011) & (df['current_year'] <= 2020)) |
    ((df['next_year'] >= 2011) & (df['next_year'] <= 2020))
]

print(f"\nNumber of files with years 2001-2010: {len(files_2001_2009)}")
print("\nBreakdown by year:")
print("\nCurrent filings:")
print(files_2001_2009['current_year'].value_counts().sort_index())
print("\nNext filings:")
print(files_2001_2009['next_year'].value_counts().sort_index())

# Number of files with years 2001-2010: 10340
# Number of files with years 2011-2020: 18696


Number of files with years 2001-2010: 18696

Breakdown by year:

Current filings:
current_year
2009       7
2010    1521
2011    1634
2012    1672
2013    1751
2014    1812
2015    1900
2016    2009
2017    2079
2018    2164
2019    2139
2020       8
Name: count, dtype: int64

Next filings:
next_year
2011    1533
2012    1631
2013    1664
2014    1759
2015    1811
2016    1905
2017    2000
2018    2083
2019    2165
2020    2145
Name: count, dtype: int64


In [5]:
# Read the paired MDA reports file in chunks to avoid memory issues
paired_file = r'C:\Users\abbra\Documents\Research\Koval Paper\Data\Output\paired_mda_reports.csv'
chunk_size = 10000

# Initialize empty sets to store unique dates
current_dates = set()
next_dates = set()

# Initialize dictionaries to store date counts
current_date_counts = {}
next_date_counts = {}

print("\nProcessing file in chunks...")

# Process file in chunks
for chunk in pd.read_csv(paired_file, chunksize=chunk_size):
    # Print column names from first chunk
    if len(current_dates) == 0:
        print("\nAvailable columns:")
        print(chunk.columns)
    
    try:
        # Update unique dates
        current_dates.update(chunk['current_date'].unique())
        next_dates.update(chunk['next_date'].unique())
        
        # Update date counts
        curr_counts = chunk['current_date'].value_counts()
        next_counts = chunk['next_date'].value_counts()
        
        for date, count in curr_counts.items():
            current_date_counts[date] = current_date_counts.get(date, 0) + count
            
        for date, count in next_counts.items():
            next_date_counts[date] = next_date_counts.get(date, 0) + count

    except KeyError as e:
        print(f"\nError: Column {str(e)} not found in the dataframe")
        print("Please check the actual column names above and update the code accordingly")
        break

# Print results
print(f"\nNumber of unique dates:")
print(f"Current MDAs: {len(current_dates)}")
print(f"Next MDAs: {len(next_dates)}")

print("\nNumber of MDAs per date (current):")
for date in sorted(current_date_counts.keys()):
    print(f"{date}: {current_date_counts[date]}")

print("\nNumber of MDAs per date (next):")
for date in sorted(next_date_counts.keys()):
    print(f"{date}: {next_date_counts[date]}")



Processing file in chunks...

Available columns:
Index(['current_filename', 'next_filename', 'company_name', 'cik_number',
       'current_filing_date', 'next_filing_date', 'current_mda_content',
       'next_mda_content', 'time_difference'],
      dtype='object')

Error: Column 'current_date' not found in the dataframe
Please check the actual column names above and update the code accordingly

Number of unique dates:
Current MDAs: 0
Next MDAs: 0

Number of MDAs per date (current):

Number of MDAs per date (next):
